# Fine Tuning with Mistral, QLora, and PEFt

 ### ResNet-50 Image Classification

This notebook demonstrates the process of fine-tuning a language model using Mistral with QLora and PEFt enhancements. We will go through the setup, configuration, and execution of the fine-tuning process, and evaluate the performance of the fine-tuned model.

## Setup

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np

## Configuration

In [ ]:
# Load the pretrained ResNet-50 model (ImageNet weight)
model = ResNet50(weights='imagenet')

## Fine-tuning Process

In [ ]:
# Path to the image you want to classify
img_path = '/content/dog.jpeg'

In [ ]:
# Load the image and convert it to the format expected by ResNet-50
img = image.load_img(img_path, target_size=(224, 224))

In [ ]:
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0) # add a batch dimension
x = preprocess_input(x) # apply model-specific preprocessing

## Evaluation

In [ ]:
# Make a prediction
preds = model.predict(x)

In [ ]:
preds

In [ ]:
# Decode the top-5 predictions
for _, label, prob in decode_predictions(preds, top=10)[0]:
    print(f"{label}: {prob:.4f}")  # fixed typo: lable -> label and corrected indentation

### BERT for Masked Language Modeling

In [ ]:
# Install Transformer if not install already
# !pip install --upgrade transformers torch --quiet
from transformers import BertTokenizer, BertForMaskedLM
import torch
import torch.nn.functional as F

In [ ]:
# Load tokenizer + pre_trained BERT (MLMhead included)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

In [ ]:
# input text (use [MASK] where we want BERT to predict a token)
text = "The capital city of France is [MASK]."

In [ ]:
# Preprocess (tokenize + convert to tensors)
inputs = tokenizer(text, return_tensors="pt") # batch of size 1

In [ ]:
# Forward pass (no gradients needed for inference)
with torch.inference_mode():
    logits = model(**inputs).logits # shape: [batch, seq_len, vocab]

In [ ]:
# Find the position of the [MASK] token
mask_idx = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

In [ ]:
# Soft-max over vocab + take top-k predictions for that position
probs = F.softmax(logits[0, mask_idx], dim=-1) # [1, vocab]
top_k = torch.topk(probs, k=5, dim=-1) # top-5 guesses

In [ ]:
# Decode and print
print("Top-5 predictions for [MASK]:")
for token_id, prob in zip(top_k.indices[0], top_k.values[0]):
    token = tokenizer.decode([token_id])  # fixed: docode -> decode and corrected brackets
    print(f"{token:<12} -> {prob.item():.4f}")  # fixed: bracket mismatch

###  BERT Sentiment Classification

In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
# Model name: BERT fine-tuned for classification
model_name = "textattack/bert-base-uncased-SST-2"

In [ ]:
# Load manually
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
# Predict
result = classifier("The movie was masterpiece. I loved it!")
print(result)

### Tokenizer Output

In [ ]:
from transformers import BertTokenizer
tok = BertTokenizer.from_pretrained("bert-base-uncased")  # fixed: "bert-base_uncased" -> "bert-base-uncased"
output = tok("I love you", return_tensors="pt")
print(output)

### BERT Tokenizer with GPT2 Model (Mismatch Warning)

In [ ]:
# Text->Token->Embedding->Model input
from transformers import BertTokenizer, GPT2Model
tok = BertTokenizer.from_pretrained("bert-base-uncased")
model = GPT2Model.from_pretrained("gpt2")
ids = tok("hello world", return_tensors="pt")["input_ids"]
model(ids)  # RuntimeError: size mismatch in Embedding layer